給定一系列城市和每對城市之間的距離

求解訪問每一座城市一次並回到起始城市的最短迴路

在TSP問題中, 兩座城市之間的距離通常是相等[對稱式]的!

給定5個城市和每對城市之間的距離

求解訪問每一座城市一次並回到起始城市的最短迴路

基因編碼 (chromosome representation)

使用城市編號來表示每一個基因, 0代表城市A, 4城市E

一開始先用亂數來產生解, 記得要回到原點

適應函數 (fitness function)

對路線進行cost (適應值)的計算

而對非法狀態列出懲罰, 將其適應度設成99999

選擇運算子與輪盤賭博演算法

對每個個體的適應值進行正規化用以算出累積機率

1. 將所有適應值/1, 並加總 = 0.14

2. 個體的適應值 / 總和 可得累加機率 

選擇運算子與輪盤賭博演算法

對隨機產生出一個 [0-1]的亂數就可以得知那一個 個體被選擇了


個體|1|2|3|4|5
-|-|-|-|-|-
1|0|7|10|2|6
2|7|0|4|9|3
3|10|4|0|5|1
4|2|9|5|0|8
5|6|3|1|8|0


In [1]:
import random
import numpy as np
import pandas as pd

path_length = {1: [ 0,  7, 10,  2,  6], 
               2: [ 7,  0,  4,  9,  3], 
               3: [10,  4,  0,  5,  1], 
               4: [ 2,  9,  5,  0,  8], 
               5: [ 6,  3,  1,  8,  0]}

population_size = 3  #族群大小
selection_rate = 0.5 #選擇率
crossover_rate = 0.8  #交配率
mutation_rate = 0.2  #突變率
epoch = 3


#=========================================
city = [1, 2, 3, 4, 5]
population = []
for i in range(population_size):
    path = random.sample(city, k=5)
    complete_path = path
    complete_path.append(path[0])
    population.append(complete_path)
population = np.array(population) #初始族群

In [2]:
def fitness_function(chromosome=None, data=None):
    """輸入一組染色體求適應值"""
    path_length_sum = 0
    for i, c in enumerate(chromosome):
        if i == 0:
            start = c
            continue
        end = c
#         print(start, "到", end, "路徑長:",data[start][end-1])
        path_length_sum += data[start][end-1]
        start = c
    fitness_value = 1/path_length_sum
    return fitness_value

def Roulette(data=None):
    """輸入族群給出輪盤選中的個體"""
    import random
    import numpy as np
    r = random.uniform(0, 1)
    for i in range(len(np.array(data))):
        if sum((np.array(data)/sum(np.array(data)))[0:i+1]) > r:
            break
    return i

def BPX_crossover(chromosome_1, chromosome_2, point=2):
    """輸入兩組染色體做交配"""
    import random
    p = random.sample(range(1,len(chromosome_1)), k=point)
    chromosome_1 = chromosome_1[:-1]
    chromosome_2 = chromosome_2[:-1]
    point_index1 = []
    point_index2 = []
#     print(p)

    for i in p:
        index = int(np.argwhere(chromosome_1 == [i]))
        point_index1.append(index) 
    for i in p:
        index = int(np.argwhere(chromosome_2 == [i]))
        point_index2.append(int(np.argwhere(chromosome_2 == [i])))    

    point_index1.sort()
    point_index2.sort()

    value = []
    for i in point_index1:
        value.append(chromosome_1[i])

    for i, j in zip(point_index2, value):
        chromosome_2[i] = j
    new_chromosome = np.append(chromosome_2, chromosome_2[0])
    return new_chromosome

def n_opt_mutation(chromosome, point=2):
    """輸入一組染色體做突變"""
    chromosome = chromosome[:-1]
    p = random.sample(range(1,len(chromosome)), k=point)
    new_chromosome = chromosome.copy()
    
    point_index = []
    for i in p:
        index = int(np.argwhere(chromosome == [i]))
        point_index.append(index) 
    value = []
    for i in point_index:
        value.append(chromosome[i])
    
    for i in point_index:
        if new_chromosome[i] == chromosome[i]:
            for v in value:
                if new_chromosome[i] != v:
                    new_chromosome[i] = v
                    value.remove(v)
    new_chromosome = np.append(new_chromosome, new_chromosome[0])
    return new_chromosome

#### 演化

In [3]:
counter = 0
while counter<epoch:
    value = []
    for p in population:
        value.append(fitness_function(chromosome=p, data=path_length))
    value = np.array(value)    #適應值

    dict = {"chromosome_0":population[:,0], 
            "chromosome_1":population[:,1],
            "chromosome_2":population[:,2],
            "chromosome_3":population[:,3],
            "chromosome_4":population[:,4],
            "chromosome_5":population[:,5],
            "fitness value":value}
    df = pd.DataFrame(dict)
    sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序

    population_selection = np.stack((sort_df["chromosome_0"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_1"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_2"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_3"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_4"].values[:int(len(sort_df)*selection_rate)], 
                                     sort_df["chromosome_5"].values[:int(len(sort_df)*selection_rate)]), axis=-1)

    population_crossover = population_selection.copy()  #直接複製

    c1type = False
    c2type = False
    while len(population_crossover) < population_size:
        for i, c in enumerate(population_selection):
            if crossover_rate > random.uniform(0, 1):
                if c1type:
                    c2 = population_selection[i]
                    c2type = True
                else:
                    c1 = population_selection[i]
                    c1type = True

                if c1type and c2type:
                    new_c = BPX_crossover(c1, c2, point=2)
                    population_crossover = np.append(population_crossover,[new_c],axis=0)
                    if len(population_crossover) >= population_size:
                        break
                    c1type = False
                    c2type = False

                else:
                    pass
        if len(population_crossover) >= population_size:
            break

    population_mutation = population_crossover.copy()

    for i, c in enumerate(population_mutation[int(population_size*selection_rate):], int(population_size*selection_rate)):  
        if mutation_rate > random.uniform(0, 1):
            population_mutation[i] = n_opt_mutation(c, point=2)

    population = population_mutation.copy()
    counter += 1
    print(counter,max(value))

1 0.05555555555555555
2 0.05555555555555555
3 0.05555555555555555


In [4]:
value = []
for p in population:
    value.append(fitness_function(chromosome=p, data=path_length))
value = np.array(value)    #適應值

dict = {"chromosome_0":population[:,0], 
        "chromosome_1":population[:,1],
        "chromosome_2":population[:,2],
        "chromosome_3":population[:,3],
        "chromosome_4":population[:,4],
        "chromosome_5":population[:,5],
        "fitness value":value}
df = pd.DataFrame(dict)
sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序
sort_df

,chromosome_0,chromosome_1,chromosome_2,chromosome_3,chromosome_4,chromosome_5,fitness value
0,1,2,5,3,4,1,0.055556
1,1,2,5,3,4,1,0.055556
2,3,2,5,1,4,3,0.050000
